In [ ]:
from leap_ec.problem import FunctionProblem
from pyhms.config import TreeConfig, CMALevelConfig, EALevelConfig
from pyhms.tree import DemeTree
from pyhms.demes.single_pop_eas.sea import SimpleEA, SEA
from pyhms.sprout.sprout_mechanisms import SproutMechanism
from pyhms.sprout.sprout_filters import NBC_FarEnough, DemeLimit, LevelLimit
from pyhms.sprout.sprout_generators import NBC_Generator
from pyhms.stop_conditions.usc import DontStop, MetaepochLimit
from pyhms.problem import EvalCutoffProblem
from pyhms.stop_conditions.gsc import SingularProblemEvalLimitReached
import pandas as pd
import numpy as np
from distribution_optimization_py.problem import (
    ScaledGaussianMixtureProblem,
)
import random
from leap_ec.representation import Representation
from distribution_optimization_py.ga_style_operators import GAStyleEA
from experiments.datasets import DATASETS, Solution
from distribution_optimization_py.gaussian_mixture import GaussianMixture

In [ ]:
config = {
    "nbc_cut": 3.7963357241892517,
    "nbc_trunc": 0.7854441324546146,
    "nbc_far": 3.698180094782821,
    "level_limit": 4,
    "pop1": 49,
    "p_mutation1": 0.21707882439727,
    "p_crossover1": 0.20534306545391615,
    "k_elites1": 1,
    "generations1": 5,
    "generations2": 26,
    "metaepoch2": 32,
    "sigma2": 0.11230013378964185,
    "use_warm_start": False
}


dataset_to_solutions = {}
for dataset in DATASETS:
    dataset_to_solutions[dataset.name] = {}
    for budget in [1000, 2000, 5000, 10000, 15000]:
        dataset_to_solutions[dataset.name][budget] = []
        for seed in range(1, 31):
            np.random.seed(seed)
            random.seed(seed)
            options = {"random_seed": seed}
            problem = ScaledGaussianMixtureProblem(dataset.data, dataset.nr_of_modes)
            bounds = np.column_stack([problem.lower, problem.upper])
            function_problem = FunctionProblem(problem, maximize=False)
            cutoff_problem = EvalCutoffProblem(function_problem, budget)
            levels_config = [
                EALevelConfig(
                    ea_class=GAStyleEA,
                    generations=config["generations1"],
                    problem=cutoff_problem,
                    bounds=bounds,
                    pop_size=config["pop1"],
                    lsc=DontStop(),
                    k_elites=config["k_elites1"],
                    representation=Representation(initialize=problem.initialize),
                    p_mutation=config["p_mutation1"],
                    p_crossover=config["p_crossover1"],
                    use_warm_start=config["use_warm_start"],
                ),
                CMALevelConfig(
                    problem=cutoff_problem,
                    bounds=bounds,
                    lsc=MetaepochLimit(config["metaepoch2"]),
                    sigma0=config["sigma2"],
                    generations=config["generations2"],
                    random_state=seed,
                ),
            ]

            sprout = SproutMechanism(
                NBC_Generator(config["nbc_cut"], config["nbc_trunc"]),
                [NBC_FarEnough(config["nbc_far"], 2), DemeLimit(1)],
                [LevelLimit(config["level_limit"])],
            )
            gsc = SingularProblemEvalLimitReached(budget)
            tree_config = TreeConfig(levels_config, gsc, sprout, options)
            deme_tree = DemeTree(tree_config)
            deme_tree.run()
            best_solution = deme_tree.best_individual
            scaled_solution = problem.reals_to_internal(best_solution.genome)
            solution = Solution(
                best_solution.fitness,
                best_solution.genome,
                scaled_solution,
            )
            dataset_to_solutions[dataset.name][budget].append(solution)

In [ ]:
for dataset_name in dataset_to_solutions:
    for budget in dataset_to_solutions[dataset_name]:
        solutions = np.array(
            [x.scaled_genome for x in dataset_to_solutions[dataset_name][budget]]
        )
        np.savetxt(f"results_hms/{dataset_name}_solutions_{budget}.csv", solutions, delimiter=",")